In [ ]:
import pandas as pd
import random
from datetime import datetime, timedelta


num_rows = 1050  
start_date = datetime(2024, 7, 1)
end_date = datetime(2025, 7, 30)


names = ["Ramesh", "Kavya", "John", "Priya", "Amit", "Sara", "David", "Meena", "Vikram", "Anita"]
blood_groups = ["A+", "A-", "B+", "B-", "O+", "O-", "AB+", "AB-"]
locations = ["Campus A", "Campus B", "Campus C", "Community Hall", "City Center"]


data = []
for _ in range(num_rows):
    date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    name = random.choice(names) + str(random.randint(1, 200)) 
    blood_group = random.choice(blood_groups)
    location = random.choice(locations)
    data.append([date.strftime("%Y-%m-%d"), name, blood_group, location])


df = pd.DataFrame(data, columns=["Date", "Name", "Blood Group", "Location"])


df.to_excel("blood_donation_1050rows.xlsx", index=False)

print("Dataset generated and saved as blood_donation_1050rows.xlsx")

In [ ]:
import pandas as pd

# Load the cleaned data
df = pd.read_excel("cleaned_donors.xlsx", sheet_name="Raw Data")

# Ensure 'Date' column is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

#Summary Statistics
summary = {
    "Total Donors": [len(df)],
    "Unique Locations": [df['Location'].nunique()],
    "Date Range": [f"{df['Date'].min().date()} to {df['Date'].max().date()}"],
    "Gender Distribution": [df['Gender'].value_counts().to_dict()]
}
summary_df = pd.DataFrame(summary)

#Pivot Table: Donors per Day
daily_pivot = df.groupby('Date').size().reset_index(name='Donor Count')

#Pivot Table: Donors per Location
location_pivot = df.groupby('Location').size().reset_index(name='Donor Count')

#Pivot Table: Donors by Gender and Location
gender_location_pivot = pd.pivot_table(df, index='Location', columns='Gender', aggfunc='size', fill_value=0)

#Pivot Table: Donors by Date and Gender
gender_date_pivot = pd.pivot_table(df, index='Date', columns='Gender', aggfunc='size', fill_value=0)

#Save to Excel
with pd.ExcelWriter("donor_dashboard.xlsx") as writer:
    df.to_excel(writer, sheet_name="Raw Data", index=False)
    summary_df.to_excel(writer, sheet_name="Summary", index=False)
    daily_pivot.to_excel(writer, sheet_name="Donors per Day", index=False)
    location_pivot.to_excel(writer, sheet_name="Donors per Location", index=False)
    gender_location_pivot.to_excel(writer, sheet_name="By Gender & Location")
    gender_date_pivot.to_excel(writer, sheet_name="By Gender & Date")

print("✅ Dashboard exported to donor_dashboard.xlsx")

In [ ]:
import pandas as pd

INPUT = "blood_donation_1050rows.xlsx"
CLEANED = "cleaned_donors.xlsx"


df = pd.read_excel(INPUT, engine="openpyxl")


df.columns = [c.strip() for c in df.columns]


required = {"Date", "Name", "Blood Group", "Location"}
if not required.issubset(df.columns):
    raise SystemExit(f"Missing columns: {required - set(df.columns)}")

df = df.dropna(subset=list(required)).copy()


df["Date"] = pd.to_datetime(df["Date"])
df["Blood Group"] = df["Blood Group"].astype(str).str.strip().str.upper()
df["Location"] = df["Location"].astype(str).str.strip().str.title()
df["Name"] = df["Name"].astype(str).str.strip().str.title()


df = df.drop_duplicates(subset=["Name", "Date", "Blood Group", "Location"])


df["DonorID"] = pd.factorize(df["Name"])[0] + 1
df["Month"] = df["Date"].dt.to_period('M').dt.to_timestamp()
df["DayOfWeek"] = df["Date"].dt.day_name()


df.to_excel(CLEANED, index=False)
print(f"✅ Cleaned & enriched data saved as: {CLEANED}")

In [ ]:
import pandas as pd

# Load cleaned data
df = pd.read_excel("cleaned_donors.xlsx", engine="openpyxl")

# Create Pivot Table: location vs number of donors
pivot_location = pd.pivot_table(
    df,
    index="Location",
    values="DonorID",  # can be any column, we'll count unique donors
    aggfunc="count"
).reset_index()

pivot_location.rename(columns={"DonorID": "Total Donors"}, inplace=True)

# Save Pivot Table to Excel
with pd.ExcelWriter("location_summary.xlsx", engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name="Cleaned Data", index=False)
    pivot_location.to_excel(writer, sheet_name="Location Summary", index=False)

print("✅ Location-wise summary saved to location_summary.xlsx")

In [ ]:
import pandas as pd
import sqlite3

CLEANED = "cleaned_donors.xlsx"
DB = "donors.db"
SQL_EXPORT = "sql_outputs.xlsx"

# 1) Load cleaned data
df = pd.read_excel(CLEANED, engine="openpyxl")

# 2) Create / overwrite SQLite DB and load table
with sqlite3.connect(DB) as conn:
    df[["Date","Name","Blood Group","Location"]].to_sql("donors", conn, if_exists="replace", index=False)

    # 3) Run helpful aggregation queries
    donors_by_group = pd.read_sql_query(
        "SELECT [Blood Group] AS BloodGroup, COUNT(*) AS Donors FROM donors GROUP BY [Blood Group] ORDER BY Donors DESC",
        conn
    )

    donors_by_location = pd.read_sql_query(
        "SELECT Location, COUNT(*) AS Donors FROM donors GROUP BY Location ORDER BY Donors DESC",
        conn
    )

    daily_counts = pd.read_sql_query(
        "SELECT Date, COUNT(*) AS Donors FROM donors GROUP BY Date ORDER BY Date",
        conn, parse_dates=["Date"]
    )

    monthly_counts = pd.read_sql_query(
        "SELECT strftime('%Y-%m', Date) AS Month, COUNT(*) AS Donors FROM donors GROUP BY strftime('%Y-%m', Date) ORDER BY Month",
        conn
    )

# 4) Save the SQL outputs as an Excel file (evidence)
with pd.ExcelWriter(SQL_EXPORT, engine="xlsxwriter") as writer:
    donors_by_group.to_excel(writer, sheet_name="By_Blood_Group", index=False)
    donors_by_location.to_excel(writer, sheet_name="By_Location", index=False)
    daily_counts.to_excel(writer, sheet_name="Daily_Counts", index=False)
    monthly_counts.to_excel(writer, sheet_name="Monthly_Counts", index=False)

print("✅ donors.db and sql_outputs.xlsx created.")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt

# Load the cleaned data
df = pd.read_excel("cleaned_donors.xlsx", sheet_name="Raw Data")

# Feature Engineering
df['DayOfWeek'] = pd.to_datetime(df['Date']).dt.dayofweek  # Monday = 0
df['Month'] = pd.to_datetime(df['Date']).dt.month

# Group by date to get donor count per day
daily_donors = df.groupby(['Date', 'DayOfWeek', 'Month', 'Location']).size().reset_index(name='DonorCount')

# Convert categorical 'Location' into numeric using one-hot encoding
daily_donors = pd.get_dummies(daily_donors, columns=['Location'])

# Features and Target
X = daily_donors.drop(columns=['Date', 'DonorCount'])
y = daily_donors['DonorCount']

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluation
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"R2 Score: {r2:.2f}")

# Plot actual vs predicted
plt.scatter(y_test, y_pred)
plt.xlabel("Actual Donors")
plt.ylabel("Predicted Donors")
plt.title("Actual vs Predicted Donors")
plt.grid(True)
plt.show()

In [ ]:
import pandas as pd

# Load your Excel file
df = pd.read_excel("cleaned_donors.xlsx", sheet_name="Raw Data")

# Add features
df['Weekday'] = pd.to_datetime(df['Date']).dt.day_name()
df['DayOfWeek'] = pd.to_datetime(df['Date']).dt.dayofweek  # Required for next line
df['IsWeekend'] = df['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)

# Save the updated data (optional)
df.to_excel("enhanced_donors.xlsx", index=False)

print("✅ Features added and file saved as 'enhanced_donors.xlsx'")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt

# Load the cleaned data
df = pd.read_excel("cleaned_donors.xlsx", sheet_name="Raw Data")

# Feature Engineering
df['Date'] = pd.to_datetime(df['Date'])
df['DayOfWeek'] = df['Date'].dt.dayofweek  # Monday = 0
df['Month'] = df['Date'].dt.month
df['IsWeekend'] = df['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)

# Group by date & location to get donor count per day
daily_donors = df.groupby(['Date', 'DayOfWeek', 'Month', 'IsWeekend', 'Location']).size().reset_index(name='DonorCount')

# Convert categorical 'Location' into numeric using one-hot encoding
daily_donors = pd.get_dummies(daily_donors, columns=['Location'])

# Features and Target
X = daily_donors.drop(columns=['Date', 'DonorCount'])
y = daily_donors['DonorCount']

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluation
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"R2 Score: {r2:.2f}")

# Plot actual vs predicted
plt.scatter(y_test, y_pred)
plt.xlabel("Actual Donors")
plt.ylabel("Predicted Donors")
plt.title("Actual vs Predicted Donors")
plt.grid(True)
plt.show()

# Save predicted vs actual to Excel
result_df = pd.DataFrame({
    'Actual Donors': y_test.values,
    'Predicted Donors': y_pred
})
result_df.to_excel("predicted_vs_actual.xlsx", index=False)

In [ ]:
import pandas as pd
import sqlite3

# Paths
INPUT = "blood_donation_1050rows.xlsx"
CLEANED = "cleaned_donors.xlsx"
DB = "donors.db"
SQL_EXPORT = "sql_outputs.xlsx"
DASHBOARD = "donor_dashboard.xlsx"

# Step 1: Clean and Enrich
df = pd.read_excel(INPUT, engine="openpyxl")
df.columns = df.columns.str.strip()
required = {"Date", "Name", "Blood Group", "Location"}
if not required.issubset(df.columns):
    raise SystemExit(f"Missing: {required - set(df.columns)}")

df = df.dropna(subset=list(required)).copy()
df["Date"] = pd.to_datetime(df["Date"])
df["Blood Group"] = df["Blood Group"].astype(str).str.strip().str.upper()
df["Location"] = df["Location"].astype(str).str.strip().str.title()
df["Name"] = df["Name"].astype(str).str.strip().str.title()
df = df.drop_duplicates(subset=["Name", "Date", "Blood Group", "Location"])
df["DonorID"] = pd.factorize(df["Name"])[0] + 1
df["Month"] = df["Date"].dt.to_period('M').dt.to_timestamp()
df["DayOfWeek"] = df["Date"].dt.day_name()
df.to_excel(CLEANED, index=False)

print("Cleaned data saved.")

# Step 2: SQL Loading and Aggregation
with sqlite3.connect(DB) as conn:
    df[["Date", "Name", "Blood Group", "Location"]].to_sql("donors", conn, if_exists="replace", index=False)
    queries = {
        "By_Blood_Group": "SELECT [Blood Group] AS BloodGroup, COUNT(*) AS Donors FROM donors GROUP BY [Blood Group] ORDER BY Donors DESC",
        "By_Location": "SELECT Location, COUNT(*) AS Donors FROM donors GROUP BY Location ORDER BY Donors DESC",
        "Daily_Counts": "SELECT Date, COUNT(*) AS Donors FROM donors GROUP BY Date ORDER BY Date",
        "Monthly_Counts": "SELECT strftime('%Y-%m', Date) AS Month, COUNT(*) AS Donors FROM donors GROUP BY strftime('%Y-%m', Date) ORDER BY Month"
    }
    result_dfs = {name: pd.read_sql_query(sql, conn, parse_dates=["Date"] if "Date" in name else None) for name, sql in queries.items()}

print("SQL queries executed.")

# Step 3: Save SQL results to Excel
with pd.ExcelWriter(SQL_EXPORT, engine="xlsxwriter") as writer:
    for sheet, rdf in result_dfs.items():
        rdf.to_excel(writer, sheet_name=sheet, index=False)

print("SQL outputs exported.")

# Step 4: Generate Dashboard (PivotTables, Summary)
# Reuse your o2_excel_dashboard or insights scripts...

# Step 5: Add Insights and Ethical Notes
# (Assuming those scripts run here)

print("Full workflow completed.")

In [ ]:
import pandas as pd

df = pd.read_csv("cleaned_donors.csv")  # replace with correct path

print(df.shape)
print(summary_df.shape)
print(daily_pivot.shape)
print(location_pivot.shape)
print(gender_location_pivot.shape)
print(gender_date_pivot.shape)
print(result_df.shape)